## Day 17: Spinlock

http://adventofcode.com/2017/day/17

### Part 1

There's better code in the post-mortem. Please don't look at this.

In [1]:
from pyrsistent import pvector 
from collections import namedtuple


Spinlock = namedtuple('Spinlock', 'buffer position')


def step_forward(spinlock, n=1):
    return Spinlock(spinlock.buffer, (spinlock.position + n) % len(spinlock.buffer))


def insert(spinlock, insertion):
    b = spinlock.buffer[:spinlock.position + 1] \
        .append(insertion) \
        .extend(spinlock.buffer[spinlock.position + 1:])
    return Spinlock(b, spinlock.position + 1)


def print_spinlock(sp):
    return ' '.join([str(x) if i != sp.position else '(' + str(x) + ')' 
                     for i, x in enumerate(sp.buffer)])

In [2]:
def run_2017(spinlock, step_number):
    for i in range(1, 2018):
        spinlock = insert(step_forward(spinlock, step_number), i)
        
    return spinlock

In [3]:
starting_spinlock = Spinlock(pvector([0]), 0)

%time test_answer = run_2017(starting_spinlock, 3)

CPU times: user 52.4 ms, sys: 178 µs, total: 52.6 ms
Wall time: 52.5 ms


In [4]:
print_spinlock(test_answer)

'0 1226 1635 517 218 920 1636 1227 690 388 1637 291 1228 921 1638 518 691 1229 1639 16 922 92 1640 1230 389 692 1641 923 1231 519 1642 164 292 1232 1643 924 693 219 1644 1233 520 925 1645 390 1234 694 1646 123 926 1235 1647 69 521 695 1648 1236 927 293 1649 391 1237 29 1650 928 696 1238 1651 522 220 929 1652 1239 165 697 1653 392 1240 930 1654 523 294 1241 1655 698 931 52 1656 1242 124 524 1657 932 1243 699 1658 393 221 1244 1659 933 295 700 1660 1245 525 934 1661 93 1246 394 1662 701 935 1247 1663 166 526 39 1664 1248 936 702 1665 296 1249 395 1666 937 527 1250 1667 703 222 938 1668 1251 70 125 1669 704 1252 939 1670 528 396 1253 1671 297 940 705 1672 1254 167 529 1673 941 1255 223 1674 706 397 1256 1675 942 12 530 1676 1257 707 943 1677 298 1258 94 1678 398 944 1259 1679 708 531 22 1680 1260 945 224 1681 709 1261 299 1682 946 532 1262 1683 399 710 947 1684 1263 168 126 1685 533 1264 948 1686 711 53 1265 1687 400 949 300 1688 1266 712 534 1689 950 1267 225 1690 71 713 1268 1691 951 40

That does work but it took a bit of finding.

In [5]:
def answer(sp):
    return sp.buffer[(sp.position + 1) % len(sp.buffer)]

answer(test_answer)

638

Now for the given problem.

In [6]:
answer(run_2017(starting_spinlock, 335))

1282

### Part 2

Hmm. What am I missing here? Let's brute force it at first to see how bad it will be.

In [7]:
def run_n(spinlock, step_number, n):
    for i in range(1, n + 1):
        spinlock = insert(step_forward(spinlock, step_number), i)
        
    return spinlock

In [8]:
%time run_n(starting_spinlock, 335, 50000)

CPU times: user 42.6 s, sys: 8.24 ms, total: 42.6 s
Wall time: 42.6 s


Spinlock(buffer=pvector([0, 22413, 6905, 23932, 37648, 46660, 844, 44487, 46107, 47786, 25101, 46939, 33218, 49971, 44753, 20557, 49674, 20374, 47502, 15213, 15767, 31671, 17553, 21433, 44091, 45155, 39019, 45833, 23578, 39487, 9613, 45425, 14205, 7373, 13829, 42289, 24147, 37536, 18301, 2159, 27449, 21497, 23719, 8558, 25554, 18911, 21179, 43829, 42924, 12459, 11773, 9845, 1860, 25176, 20928, 4074, 3978, 40925, 17345, 30741, 15304, 36761, 27043, 2361, 35049, 17658, 21116, 27531, 34737, 16390, 40440, 40080, 22480, 16488, 25478, 28962, 37313, 4645, 35788, 39605, 38903, 11158, 14988, 5440, 20680, 49232, 44620, 47644, 19894, 19195, 42542, 45970, 36002, 32051, 42669, 28364, 10234, 26643, 49379, 5296, 22954, 45290, 3489, 20253, 37202, 21885, 24364, 13185, 48072, 5757, 43960, 13463, 12836, 43439, 14290, 1574, 38442, 41788, 41416, 11738, 2447, 23438, 279, 27286, 12385, 46383, 13383, 29661, 6864, 20013, 22682, 33717, 1612, 47929, 24952, 47220, 4235, 10481, 3051, 2988, 23508, 16439, 2418, 46245

In [9]:
42000/60/24

29.166666666666668

About a month assuming linear growth, which is unlikely.

I could use a more efficient data structure for the buffer but hang on, I don't actually have to build the buffer at all. Zero is always at the beginning in the representation used above, so track the insertion point and return the last number to be put after index 0.

In [10]:
def number_after_zero(step_number, n):
    result = None
    p = 0
    length_of_buffer = 1
    
    for i in range(1, n + 1):
        # Length at each step is i
        p = ((p + step_number) % i) + 1
        if p == 1:
            result = i
            
    return result

In [11]:
%time number_after_zero(335, 50000000)

CPU times: user 5.6 s, sys: 0 ns, total: 5.6 s
Wall time: 5.6 s


27650600

Hurrah!

### Post-mortem

Just out of interest, let's have a go with pyrsistent's deque to see if a brute force is possible.

In [12]:
from pyrsistent import pdeque


def p_run_n(spinlock, step_number, n):
    for i in range(1, n + 1):
        spinlock = spinlock.rotate(-step_number-1).appendleft(i)
        
    return spinlock

In [18]:
# DON'T RUN THIS %time p_run_n(pdeque([0]), 335, 50000)

There seems to be too much overhead with the persistent deque. Let's try good old mutable deque from collections.

In [14]:
from collections import deque

def d_run_n(spinlock, step_number, n):
    for i in range(1, n + 1):
        spinlock.rotate(-step_number-1)
        spinlock.appendleft(i)
        
    return spinlock

In [15]:
%time d_run_n(deque([0]), 335, 50000)

CPU times: user 29 ms, sys: 1 µs, total: 29 ms
Wall time: 29.3 ms


deque([50000,
       42062,
       45587,
       31974,
       3491,
       24892,
       26579,
       4471,
       24161,
       29239,
       1850,
       11571,
       26264,
       44779,
       32551,
       34141,
       10487,
       6909,
       43464,
       37446,
       30034,
       24597,
       31784,
       3071,
       29152,
       28044,
       11468,
       16302,
       22493,
       344,
       21318,
       14687,
       14003,
       33636,
       2405,
       35174,
       49555,
       35279,
       21963,
       40343,
       30576,
       38579,
       30485,
       13232,
       2219,
       41071,
       6432,
       39510,
       9763,
       7182,
       13511,
       18922,
       3829,
       34243,
       26738,
       48822,
       20569,
       11815,
       6206,
       24818,
       2775,
       49703,
       37670,
       43724,
       27465,
       17721,
       44513,
       42949,
       35490,
       40223,
       34965,
       20692,
       

That's a lot better and the code is quite a bit nicer than the original, mutability aside. Let's have a bash at the fifty million.

In [16]:
%time answer = d_run_n(deque([0]), 335, 50000000)

CPU times: user 51.8 s, sys: 908 ms, total: 52.7 s
Wall time: 52.7 s


In [17]:
answer[answer.index(0) + 1]

27650600

I'm actually quite impressed that a naive brute force in Python takes less than a minute. Pypy takes five times longer.